In [2]:
import numpy as np

In [3]:
def rastrigin(X):
    if isinstance(X[0], (int, float)):
        X = [[X[i]] for i in range(len(X))]

    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

def sphere(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = sum(np.array(xi) ** 2)
        val.append(fx)
    return np.array(val)

def rosenbrock(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = sum(100 * (np.array(xi[1:]) - np.array(xi[:-1]) ** 2) ** 2 + (1 - np.array(xi[:-1])) ** 2)
        val.append(fx)
    return np.array(val)


def ackley(X):
    if isinstance(X[0],(int,float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = -20 * np.exp(-0.2 * np.sqrt(sum(np.array(xi) ** 2) / len(xi))) - np.exp(sum(np.cos(2 * np.pi * np.array(xi))) / len(xi)) + 20 + np.exp(1)
        val.append(fx)
    return np.array(val)

In [4]:
def generate_particle(num_variables,swarm_size,x_min,x_max):
    swarm = np.random.uniform(x_min,x_max,(swarm_size,num_variables))

    velocity = np.zeros_like(swarm)

    return swarm , velocity

In [5]:
def pso2(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon, function=rastrigin):

    swarm, velocity = generate_particle(num_variables, swarm_size, x_min, x_max)
    swarm = np.clip(swarm, x_min, x_max)
    swarm_positions = [swarm.tolist()]

    for i in range(num_iterations):
        # print('Iteration:', i)
        swarm2 = swarm + epsilon * velocity
        swarm2 = np.clip(swarm2, x_min, x_max)
        
        dicx = {tuple(swarm[i]): function(swarm)[i] for i in range(swarm_size)}
        dicy = {tuple(swarm2[i]): function(swarm2)[i] for i in range(swarm_size)}
        # print(dicx)

        if i == 0:
            local_best = swarm
            global_best = list(min(dicx, key=dicx.get))
        else:
            for j in range(swarm_size):
                key1, value1 = list(dicx.items())[j]
                key2, value2 = list(dicy.items())[j]
                
                local_best[j] = list(key2) if value2 < value1 else list(key1)
                
                global_best = list(min(dicy, key=dicy.get)) if np.any(np.array(list(dicy.values())) < np.array(list(dicx.values()))) else list(min(dicx, key=dicx.get))
        
        # print(global_best)

        velocity = ((alpha * (alpha - 0.4) * velocity * i) / num_iterations) + \
                   (np.random.uniform(0, beta) * (np.array(local_best) - np.array(swarm))) + \
                   (np.random.uniform(0, gamma) * (np.full_like(swarm, global_best) - np.array(swarm)))
        
        swarm = swarm2
        swarm_positions.append(swarm.tolist())

    return global_best, swarm_positions



In [7]:
num_iterations = 200
swarm_size = 100
num_variables = 2
x_min = -5
x_max = 5
alpha = 0.9
beta = 0.2
gamma = 0.7
epsilon = 0.2


functions = [rastrigin, sphere, rosenbrock, ackley]
results = {}

for func in functions:
    global_best, swarm_positions = pso2(num_iterations, swarm_size, num_variables, x_min, x_max, alpha, beta, gamma, epsilon, function=func)
    global_best_value = func([global_best])
    results[func.__name__] = {
        'global_best': global_best,
        'global_best_value': global_best_value,
        'swarm_positions': swarm_positions
    }


for func_name, result in results.items():
    print(f"Testing Particle Swarm Optimization with: {func_name}")
    print(f"Global Best: {result['global_best']}")
    print(f"Global Best Value: {result['global_best_value']}")
    # print(f"Swarm Positions: {result['swarm_positions'][:2]}")
    print()

Testing Particle Swarm Optimization with: rastrigin
Global Best: [-1.344908055753581e-09, 2.376406622046473e-09]
Global Best Value: [0.]

Testing Particle Swarm Optimization with: sphere
Global Best: [-2.913048664423185e-12, -4.749938907398819e-12]
Global Best Value: [3.10477721e-23]

Testing Particle Swarm Optimization with: rosenbrock
Global Best: [0.8475505669968298, 0.7175626751327775]
Global Best Value: [0.02330156]

Testing Particle Swarm Optimization with: ackley
Global Best: [-3.110308272445469e-12, -3.533160664564651e-12]
Global Best Value: [1.3316015e-11]

